In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
customer_df = pd.read_csv('Wholesale_customers_data.csv')
customer_df.drop(['Channel', 'Region'], axis=1, inplace=True)

In [3]:
customer_df.shape

(440, 6)

# Correlation and Redundancy

I claim that there is correlation and redundancy in the `customer` table. What I mean by this is that some features are linear combinations of other features. 

Let's examine redundancy by dropping a feature and seeing if the other features can predict it.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [5]:
def calculate_r_2_for_feature(data,feature):
    new_data = data.drop(feature, axis=1)

    X_train, \
    X_test,  \
    y_train, \
    y_test = train_test_split(
        new_data,data[feature],test_size=0.25
    )

    regressor = KNeighborsRegressor()
    regressor.fit(X_train,y_train)

    score = regressor.score(X_test,y_test)
    return score

In [6]:
calculate_r_2_for_feature(customer_df,'Detergents_Paper')

0.6515303633904842

In [8]:
print("{:24} {}".format("Delicatessen: ", calculate_r_2_for_feature(customer_df,'Delicatessen')))
print("{:24} {}".format("Degergents_paper: ", calculate_r_2_for_feature(customer_df,'Detergents_Paper')))
print("{:24} {}".format("Fresh: ", calculate_r_2_for_feature(customer_df,'Fresh')))
print("{:24} {}".format("Frozen: ", calculate_r_2_for_feature(customer_df,'Frozen')))
print("{:24} {}".format("Grocery: ", calculate_r_2_for_feature(customer_df,'Grocery')))
print("{:24} {}".format("Milk: ", calculate_r_2_for_feature(customer_df,'Milk')))

Delicatessen:            -0.10101805779688466
Degergents_paper:        0.8064438456501545
Fresh:                   -0.031982882616610864
Frozen:                  -0.008561158737762753
Grocery:                 0.7504703913945503
Milk:                    0.5502904289178931


But this is subject to randomness. There is randomness in my `train_test_split`. Let's do the whole thing many times and take the average. 

In [9]:
def mean_r2_for_feature(data, feature):
    scores = []
    for _ in range(100):
        scores.append(calculate_r_2_for_feature(data, feature))
        
    scores = np.array(scores)
    return scores.mean()

In [11]:
print("{:24} {}".format("Delicatessen: ", mean_r2_for_feature(customer_df,'Delicatessen')))
print("{:24} {}".format("Detergents_Paper: ", mean_r2_for_feature(customer_df,'Detergents_Paper')))
print("{:24} {}".format("Fresh: ", mean_r2_for_feature(customer_df,'Fresh')))
print("{:24} {}".format("Frozen: ", mean_r2_for_feature(customer_df,'Frozen')))
print("{:24} {}".format("Grocery: ", mean_r2_for_feature(customer_df,'Grocery')))
print("{:24} {}".format("Milk: ", mean_r2_for_feature(customer_df,'Milk')))

Delicatessen:            -0.045472537525694155
Detergents_Paper:        0.768313130885257
Fresh:                   0.028243797281198454
Frozen:                  0.049498252628471545
Grocery:                 0.7232750940751902
Milk:                    0.4576987517206871


In [ ]:
print("{:24} {}".format("Delicatessen: ", mean_r2_for_feature(customer_df,'Delicatessen')))
print("{:24} {}".format("Detergents_Paper: ", mean_r2_for_feature(customer_df,'Detergents_Paper')))
print("{:24} {}".format("Fresh: ", mean_r2_for_feature(customer_df,'Fresh')))
print("{:24} {}".format("Frozen: ", mean_r2_for_feature(customer_df,'Frozen')))
print("{:24} {}".format("Grocery: ", mean_r2_for_feature(customer_df,'Grocery')))
print("{:24} {}".format("Milk: ", mean_r2_for_feature(customer_df,'Milk')))

#### Discussion

What does this tell us?

## Visualize Redundancy

Study the correlation of the data.

In [ ]:
import time
start = time.time()
sns.pairplot(customer_df, kind='reg')
print(time.time() - start)

In [ ]:
corr = customer_df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, 0)] = True
with sns.axes_style("white"):
    ax = sns.heatmap(corr, mask=mask, square=True, annot=True,
                     cmap='RdBu', fmt='+.3f')
    plt.xticks(rotation=45, ha='center')